### Atypical Combination
 - Brian Uzzi et al. ,Atypical Combinations and Scientific Impact.Science342,468-472(2013).DOI:10.1126/science.1240474

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm, trange
tqdm.pandas()
from matplotlib import pyplot as plt
import json, os, shutil
from operator import add
from glob import glob
import pickle as pkl

In [ ]:
# %time atyp_comb_2025 = pd.concat([pd.read_parquet(f"intermediate/atypical-combination-2/z_10_pct/{year}.parquet") for year in trange(1800, 1930)])
# atyp_comb_2025 = pd.concat([
#     pd.read_parquet(f"intermediate/atypical-combination-2/z_10_pct/{year}.parquet")
#     for year in trange(2011, 2015)
# ])
atyp_comb_3 = pd.read_feather("intermediate/atypical-combination-zihang-3/z_10_pct_1950.feather")
atyp_comb_3["paper_id"] = atyp_comb_3["paper_id"].str[4:].astype(int)
atyp_comb_3.columns = ['paper_id', 'atyp_comb_3', 'valid_pairs']

In [ ]:
atyp_comb_2 = pd.read_feather("intermediate/atypical-combination-2/z_10_pct_1960.feather")
atyp_comb_2["paper_id"] = atyp_comb_2["paper_id"].str[4:].astype(int)
atyp_comb_2.columns = ['paper_id', 'atyp_comb_2', 'valid_pairs']

atyp_comb_1 = pd.read_feather("intermediate/atypical-combination-21/z_10_pct_1960.feather")
atyp_comb_1["paper_id"] = atyp_comb_1["paper_id"].str[4:].astype(int)
atyp_comb_1.columns = ['paper_id', 'atyp_comb_1', 'valid_pairs']

combination = atyp_comb_1.merge(atyp_comb_2, on="paper_id", how="inner")
print(combination[["valid_pairs_x", "valid_pairs_y"]].corr())
print(combination[["atyp_comb_1", "atyp_comb_2"]].corr())

In [ ]:
combination = atyp_comb_2.merge(atyp_comb_3, on="paper_id", how="inner")
print(combination[["valid_pairs_x", "valid_pairs_y"]].corr())
print(combination[["atyp_comb_2", "atyp_comb_3"]].corr())

In [ ]:
atyp_comb_1 = pd.read_feather("/kellogg/proj/dashun/dimensions/data_dump/20230910/tsv/Metrics/Z_10_pct_Atyp_Comb.feather")
atyp_comb_1.columns = ['paper_id', 'atyp_comb_1', 'valid_pairs']

atyp_comb_2 = pd.read_feather("intermediate/atypical-combination/z_10_pct_2011.feather")
atyp_comb_2["paper_id"] = atyp_comb_2["paper_id"].str[4:].astype(int)
atyp_comb_2.columns = ['paper_id', 'atyp_comb_2', 'valid_pairs']

combination = atyp_comb_1.merge(atyp_comb_2, on="paper_id", how="inner")
print(combination[["valid_pairs_x", "valid_pairs_y"]].corr(method='spearman'))
print(combination[["atyp_comb_1", "atyp_comb_2"]].corr(method='spearman'))

In [ ]:
atyp_comb_1 = pd.read_feather("/kellogg/proj/dashun/dimensions/data_dump/20230910/tsv/Metrics/Z_10_pct_Atyp_Comb.feather")
atyp_comb_1.columns = ['paper_id', 'atyp_comb_1', 'valid_pairs']

atyp_comb_2 = pd.read_feather("intermediate/atypical-combination/z_10_pct_2011.feather")
atyp_comb_2["paper_id"] = atyp_comb_2["paper_id"].str[4:].astype(int)
atyp_comb_2.columns = ['paper_id', 'atyp_comb_2', 'valid_pairs']

combination = atyp_comb_1.merge(atyp_comb_2, on="paper_id", how="inner")
print(combination[["valid_pairs_x", "valid_pairs_y"]].corr(method='spearman'))
print(combination[["atyp_comb_1", "atyp_comb_2"]].corr(method='spearman'))

In [ ]:
import os
import pandas as pd

# Directory containing the feather files
disruptiveness_dir = "/kellogg/proj/dashun/dimensions/data_dump/20250221/erzhuo/intermediate/disruptiveness"

# Get all feather files in the directory
import glob
feather_files = glob.glob(os.path.join(disruptiveness_dir, '*.feather'))

# Read and concatenate all feather files
from tqdm import tqdm

dfs = []
for file in tqdm(feather_files):
    file_path = os.path.join(disruptiveness_dir, file)
    df = pd.read_feather(file_path)
    dfs.append(df)

# Concatenate all dataframes
disruptiveness_combined = pd.concat(dfs, ignore_index=True).reset_index(drop=True)

print(f"Total rows: {len(disruptiveness_combined)}")
print(f"Columns: {disruptiveness_combined.columns.tolist()}")
print(disruptiveness_combined.head())

In [ ]:
# Save the combined disruptiveness data
output_dir = "data"
os.makedirs(output_dir, exist_ok=True)

# Save as feather
disruptiveness_combined.to_feather(os.path.join(output_dir, "disruptiveness.feather"))

# Save as tsv
disruptiveness_combined.to_csv(os.path.join(output_dir, "disruptiveness.tsv"), sep='\t', index=False)

print(f"Saved disruptiveness data to {output_dir}/disruptiveness.feather and {output_dir}/disruptiveness.tsv")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Get the old and new z_10_pct values
values_2023 = combination["atyp_comb_2023"].values
values_2025 = combination["atyp_comb_2025"].values

# Create figure
fig, (ax1) = plt.subplots(1, 1, figsize=(8, 6))

# Function to create CDF with log scale for both positive and negative parts
def plot_cdf_symlog(ax, values, label, color):
    # Sort values
    sorted_vals = np.sort(values)
    # Create CDF y-values
    y = np.arange(1, len(sorted_vals) + 1) / len(sorted_vals)
    
    # Plot CDF
    ax.plot(sorted_vals, y, label=label, color=color, linewidth=2)
    ax.set_xscale('symlog', linthresh=4, base=2)
    ax.set_xlabel('Atypical Combination Z-score')
    ax.set_ylabel('Cumulative Probability')
    ax.grid(True, alpha=0.3)
    ax.legend()

plot_cdf_symlog(ax1, values_2023, '2023 Atypical Combination', 'blue')
plot_cdf_symlog(ax1, values_2025, '2025 Atypical Combination', 'red')
ax1.set_title('CDF Comparison: 2023 vs 2025 Atypical Combination')

plt.tight_layout()
plt.show()

# Print some summary statistics
print(f"Old values - Min: {values_2023.min():.3f}, Max: {values_2023.max():.3f}, Median: {np.median(values_2023):.3f}")
print(f"New values - Min: {values_2025.min():.3f}, Max: {values_2025.max():.3f}, Median: {np.median(values_2025):.3f}")

In [ ]:
combination